In [2]:
path = "./"

In [3]:
import pandas as pd
from numpy import argmax, array, mean
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

test_size = .2
valid_size = .1
seed = 1024

#### Load data from csv

In [4]:
user_ratings_data = pd.read_csv(f"{path}/user_ratings.csv")
users_data = pd.read_csv(f"{path}/users.csv")
roars_data = pd.read_csv(f"{path}/roars.csv")

users_preference = users_data[[f"{i}" for i in range(10)]]
users_data["main_category"] = [argmax(i) for i in users_preference.to_numpy()]

names = users_data.columns.to_list()
names[0] = "user_id"
users_data.columns = names

names = roars_data.columns.to_list()
names[0] = "roar_id"
roars_data.columns = names
del names

# merge in users_data
user_ratings_data = pd.merge(user_ratings_data, users_data, on="user_id")
user_ratings_data = pd.merge(user_ratings_data, roars_data, on="roar_id")

# fix indexes in roars
old_id = roars_data["roar_id"].copy()
roars_data["roar_id"] = [i for i in range(len(old_id))]

dict_roar_id = {}
for i, value in enumerate(old_id):
  dict_roar_id[value] = i

user_ratings_data.roar_id = user_ratings_data["roar_id"].map(dict_roar_id)

In [5]:
X_train_full, X_test, Y_train_full, Y_test = train_test_split(user_ratings_data[["user_id", "roar_id"]], user_ratings_data["rating"], test_size=test_size , random_state=seed)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_full, Y_train_full, test_size=valid_size, random_state=seed)

#### Construct recommender

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

tf.random.set_seed(seed)

2022-06-22 04:31:03.665480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-22 04:31:03.665567: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
class NeuMF:

    def __init__(self, user_num, item_num):
        """
            Neural Matrix Factorization is a neural network with the following architecture:
            
            |=========================================|
            |  Legend:                                |
            |  - Generic Matrix Factorization (GMF)   |
            |  - Multilayer Perceptron (MLP)          |
            |=========================================|
            
            
            [[GMF]      [MLP]]
            [   [Dense L]    ]
            [     [Result]   ]
        """
        latent_features = 8

        # Input
        user = Input(shape=(1,), dtype='int32')
        item = Input(shape=(1,), dtype='int32')
        
        # User embedding for GMF
        gmf_user_embedding = Embedding(user_num, latent_features, input_length=user.shape[1])(user)
        gmf_user_embedding = Flatten()(gmf_user_embedding)

        # Item embedding for GMF
        gmf_item_embedding = Embedding(item_num, latent_features, input_length=item.shape[1])(item)
        gmf_item_embedding = Flatten()(gmf_item_embedding)

        # User embedding for MLP
        mlp_user_embedding = Embedding(user_num, 32, input_length=user.shape[1])(user)
        mlp_user_embedding = Flatten()(mlp_user_embedding)

        # Item embedding for MLP
        mlp_item_embedding = Embedding(item_num, 32, input_length=item.shape[1])(item)
        mlp_item_embedding = Flatten()(mlp_item_embedding)

        # GMF layers
        gmf_mul =  Multiply()([gmf_user_embedding, gmf_item_embedding])

        # MLP layers
        mlp_concat = Concatenate()([mlp_user_embedding, mlp_item_embedding])
        mlp_dropout = Dropout(0.2)(mlp_concat)

        # Layer1
        mlp_layer_1 = Dense(units=64, activation='relu', name='mlp_layer1')(mlp_dropout)  # (64,1)
        mlp_dropout1 = Dropout(rate=0.2, name='dropout1')(mlp_layer_1)                    # (64,1)
        mlp_batch_norm1 = BatchNormalization(name='batch_norm1')(mlp_dropout1)            # (64,1)

        # Layer2
        mlp_layer_2 = Dense(units=32, activation='relu', name='mlp_layer2')(mlp_batch_norm1)  # (32,1)
        mlp_dropout2 = Dropout(rate=0.2, name='dropout2')(mlp_layer_2)                        # (32,1)
        mlp_batch_norm2 = BatchNormalization(name='batch_norm2')(mlp_dropout2)                # (32,1)

        # Layer3
        mlp_layer_3 = Dense(units=16, activation='relu', name='mlp_layer3')(mlp_batch_norm2)  # (16,1)

        # Layer4
        mlp_layer_4 = Dense(units=8, activation='relu', name='mlp_layer4')(mlp_layer_3)       # (8,1)

        # merge GMF + MLP
        merged_vector = tf.keras.layers.concatenate([gmf_mul, mlp_layer_4])

        # Output layer
        output_layer = Dense(1, kernel_initializer='lecun_uniform', name='output_layer')(merged_vector) # 1,1 / h(8,1)

        # Model
        self.model = Model([user, item], output_layer)
        self.model.compile(optimizer= 'adam', loss= 'msle')

    def get_model(self):
        model = self.model
        return model


In [8]:
roar_size = len(X_train["roar_id"].unique())
roar_size

46769

In [9]:
nmf = NeuMF(5000, roar_size)
model = nmf.get_model()

2022-06-22 04:31:35.210000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-22 04:31:35.210080: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-22 04:31:35.210139: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (atp-ariel): /proc/driver/nvidia/version does not exist
2022-06-22 04:31:35.275330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
X = [X_train["user_id"], X_train["roar_id"]]
X_v = (X_valid["user_id"], X_valid["roar_id"])
X_t = (X_test["user_id"], X_test["roar_id"])

In [11]:
model = tf.keras.models.load_model(path + "/neuMF")
model.load_weights(path + "/neuMF_w")

In [16]:
loss = [] 
val_loss = [] 

In [ ]:
checkpoints = 10
for i in range(checkpoints):
  print("Iter", i)
  temp = model.fit(X, Y_train, epochs=10, batch_size=1024, validation_data=(X_v, Y_valid))
  loss += temp.history["loss"]
  val_loss += temp.history["val_loss"]
  model.save(path + "/neuMF")
  model.save_weights(path + "/neuMF_w")
  plt.plot([i for i in range(len(loss))], loss, label="Training")
  plt.plot([i for i in range(len(val_loss))], val_loss, label="Validation")
  plt.legend()
  plt.show()

In [12]:
a = model.predict(X_t)


50932/53885 [===========================>..] - ETA: 7s

In [15]:
Y_t = Y_test.to_numpy()
Y_t.shape

(1724320,)

In [16]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()

In [19]:
e = []
for i in range(Y_t.shape[0]//30000):
  e.append(msle(Y_t[30000*i: 30000*(i+1)], a[30000*i: 30000*(i+1)]).numpy())
print(mean(e))

0.41354555
